# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import pprint as pp
from scipy.stats import linregress
from api_keys import geoapify_key



In [48]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
data = pd.read_csv("output/cities_weather.csv")
# city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
data.head()

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description
0,0,Mwinilunga,ZM,-11.7358,24.4293,17.11,2.84,1703717228,99,97,light rain
1,1,Labytnangi,RU,66.6572,66.4183,-10.12,8.48,1703717228,94,100,light snow
2,2,Adamstown,PN,-25.0660,-130.1015,22.78,7.53,1703717229,74,100,overcast clouds
3,3,Igarka,RU,67.4667,86.5833,-23.81,2.74,1703717229,95,53,broken clouds
4,4,Vilyuchinsk,RU,52.9306,158.4028,-7.82,3.54,1703717229,93,100,light snow


In [49]:
city_data_df=data.copy()
len(city_data_df)

585

In [50]:
city_data_df.dropna(subset='Country')
city_data_df.sample(10)

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description
246,246,Rāy,IN,23.6833,85.0500,13.43,0.80,1703717280,70,0,clear sky
102,102,Howard Springs,AU,-12.4970,131.0470,26.66,0.63,1703717250,99,91,overcast clouds
523,523,Pikalyovo,RU,59.5183,34.1664,-17.04,0.94,1703717339,100,99,overcast clouds
88,88,Maba,CN,24.6841,113.5984,9.67,0.83,1703717247,72,0,clear sky
518,518,La Charité-sur-Loire,FR,47.1833,3.0167,6.16,3.93,1703717338,70,99,overcast clouds
224,224,Tupanatinga,BR,-8.7533,-37.3397,27.12,5.07,1703717275,44,62,broken clouds
398,398,Jutaí,BR,-5.1833,-68.9000,24.17,0.89,1703717314,97,99,overcast clouds
570,570,Camayenne,GN,9.5350,-13.6878,27.17,3.60,1703717350,74,5,clear sky
30,30,Waitangi,NZ,-43.9535,-176.5597,12.75,4.76,1703717235,62,74,broken clouds
34,34,Māngrol,IN,21.1167,70.1167,20.96,4.59,1703717236,42,68,broken clouds


In [51]:
len(city_data_df)

585

In [52]:
city_data_df.loc[city_data_df['Country'] == 'US']
city_data_df.sample(5)

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description
537,537,Guerrero Negro,MX,27.9769,-114.0611,21.32,5.39,1703717342,33,52,broken clouds
149,149,Margaret River,AU,-33.9500,115.0667,15.89,6.01,1703717260,72,8,clear sky
93,93,Galveston,US,29.3669,-94.9669,18.35,1.79,1703717249,32,29,scattered clouds
105,105,Zamora,EC,-4.0692,-78.9567,24.58,0.46,1703717251,99,100,light rain
174,174,Lompoc,US,34.6391,-120.4579,17.55,2.57,1703717265,72,20,few clouds


In [53]:
city_data_df.columns.to_list()

['City_ID',
 'City Name',
 'Country',
 'City Lat',
 'City Lon',
 'Max Temp',
 'Wind',
 'Date',
 'Humidity',
 'Cloudiness',
 'Description']

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [54]:
df = city_data_df.loc[city_data_df['Country'].notna()]
df.sample(15)
len(df)

584

In [55]:
%%capture --no-display
# https://hvplot.holoviz.org/reference/geopandas/points.html
# Configure the map plot
map_plot = df.hvplot.points(
    "City Lon", 
    "City Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City Name",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [City Lon,City Lat]   (City Name,Humidity)

In [56]:
df.sample()

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description
237,237,Ust-Nera,RU,64.5667,143.2,-40.65,1.09,1703717278,100,98,overcast clouds


In [57]:
df.columns

Index(['City_ID', 'City Name', 'Country', 'City Lat', 'City Lon', 'Max Temp',
       'Wind', 'Date', 'Humidity', 'Cloudiness', 'Description'],
      dtype='object')

In [58]:
## set copy warning on this code
# for b in df.loc[:,'Max Temp']:
#     b= float(b * 9/5 + 32)
#     b="{:.2f}".format(b)
#     df.loc[:, ('Max Temp (f)')]= b


In [59]:
df.sample()

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description
430,430,Stephenville,US,32.2207,-98.2023,10.64,3.13,1703717320,53,9,clear sky


In [60]:
# No set copy warning
df=df.copy()
df['Max Temp (f)'] =df.apply(lambda x: (9/5)*x['Max Temp']+32,axis=1)

In [61]:
df.sample()

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description,Max Temp (f)
358,358,Kingston,JM,17.997,-76.7936,27.53,4.63,1703717202,86,75,light rain,81.554


In [62]:
# i don't think there are any null values, but let's check
nan_rows = df[df.isnull().T.any()]
nan_rows

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description,Max Temp (f)


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [63]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
prime_choice=df.loc[(df['Max Temp (f)']>= 50) & (df['Max Temp (f)'] <72)]
len(prime_choice)

182

In [64]:
# Drop any rows with null values
#not necessary

prime_choice.dropna()

len(prime_choice)

182

In [65]:
# Display sample data
prime_choice.sample(5)

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description,Max Temp (f)
17,17,Jiazi,CN,22.8779,116.0670,17.54,4.91,1703717232,82,100,overcast clouds,63.572
200,200,Illizi,DZ,26.4833,8.4667,14.89,3.84,1703717270,25,60,broken clouds,58.802
182,182,Gēdo,ET,9.0167,37.4500,14.25,1.01,1703717267,61,100,overcast clouds,57.650
20,20,Sonoita,MX,31.8500,-112.8333,19.18,2.10,1703717233,17,46,scattered clouds,66.524
508,508,Mwene-Ditu,CD,-7.0000,23.4500,18.45,1.55,1703717336,99,100,light rain,65.210


In [66]:
## Send results to csv so that if jupyter notebook fails, we don't need to run all cells again
# local folder
prime_choice.to_csv('output/curated_cities_jupyter_notebook_version.csv', index=False)


### Test api with local sample - commented out

In [67]:
# url components 
# https://api.geoapify.com/v2/places?PARAMS
# bias=proximity:lon,lat
# category
# conditions accommodation.hotel, leisure.spa
# city
# filter

# categories? internet_access, dogs.yes
# supplemental api option: https://apidocs.geoapify.com/docs/place-details/ #place-details

In [68]:
# import requests

# import pprint as pp

# # url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel,leisure.spa&filter=place:51df0bbcd7fdef55c0597f7dfc8e1fa93e40f00101f901b7351c0000000000c0020992030e42616c6477696e20436f756e7479&limit=100&apiKey={geoapify_key}"


# url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel,leisure.spa&filter=circle:-87.877546,30.694666,50000&bias=proximity:-87.877546,30.694666&limit=5&apiKey={geoapify_key}"
          
        
# response = requests.get(url)
# info=response.json()


In [69]:
# pp.pprint(info['features'][0])

In [70]:
# locality=info['features'][0]['properties']['city']
# locality

In [71]:
# country=info['features'][0]['properties']['country']
# country

In [72]:
# address=info['features'][0]['properties']['address_line2']
# address

In [73]:
# place_id=info['features'][0]['properties']['place_id']
# place_id

##  Geopify request on selection of cities meeting temperature criteria

In [74]:
# create new dataframe columns to store information 
curated_cities_df = pd.read_csv('output/curated_cities_jupyter_notebook_version.csv',index_col=False )

In [75]:
curated_cities_df.head()

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description,Max Temp (f)
0,0,Mwinilunga,ZM,-11.7358,24.4293,17.11,2.84,1703717228,99,97,light rain,62.798
1,9,Blackmans Bay,AU,-43.0167,147.3167,19.07,2.45,1703717230,78,62,broken clouds,66.326
2,13,Taoudenni,ML,22.6783,-3.9836,21.48,5.40,1703717231,10,0,clear sky,70.664
3,14,Mount Gambier,AU,-37.8333,140.7667,17.61,3.22,1703717232,65,95,overcast clouds,63.698
4,17,Jiazi,CN,22.8779,116.0670,17.54,4.91,1703717232,82,100,overcast clouds,63.572


In [76]:
santa_rosalia=curated_cities_df.loc[curated_cities_df['City Name']=='Santa Rosalía']
santa_rosalia

,City_ID,City Name,Country,City Lat,City Lon,Max Temp,Wind,Date,Humidity,Cloudiness,Description,Max Temp (f)
104,327,Santa Rosalía,MX,27.3167,-112.2833,19.99,3.93,1703717121,34,25,scattered clouds,67.982


### Step 3: Create a new DataFrame called `hotel_df`.

In [77]:
#narrow down dataframe 
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotels_df = curated_cities_df[["City Name", "Country", "City Lat", "City Lon", "Humidity"]].copy()

In [78]:
#add new columns to save the data from api

hotels_df['Hotel Name']=''
hotels_df['Hotel Address']=''



In [79]:

hotels_df.sample()

,City Name,Country,City Lat,City Lon,Humidity,Hotel Name,Hotel Address
131,Chui,UY,-33.6971,-53.4616,74,,


In [80]:
# sample alternate for setting up url
# Set the parameters for the type of search
# limit = 20
# radius = 1000
# filters = f"circle:{longitude},{latitude},{radius}"
# bias = f"proximity:{longitude},{latitude}"

# # set up a parameters dictionary
# params = {
#     "filter":filters,
#     "limit":limit,
#     "bias":bias,
#     "apiKey":geoapify_key    
# }

# # Set base URL
# base_url = "https://api.geoapify.com/v2/places"

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [82]:
for index, row in hotels_df.iterrows():
    radius=10000
    categories=f'accommodation.hotel'
    limit=20
#     print(index[0])
#     print(row['City Name'])
    city=row['City Name']
#     print(row['City Lon'])
    longitude=row['City Lon']
#     print(row['City Lat'])
    latitude=row['City Lat']
    url=f"https://api.geoapify.com/v2/places?categories={categories}&filter=circle:{longitude},{latitude},{radius}&bias=proximity:{longitude},{latitude}&limit={limit}&apiKey={geoapify_key}"
#     print(url)
    response=requests.get(url)
    info=response.json()
#     pp.pprint(info)

    try:
        name=info['features'][0]['properties']['name']
        hotels_df.loc[index,'Hotel Name']=name
        print(f'For {city}, the nearest hotel is {name}')
    except:
        print('hotel name not found, skipping')
        hotels_df.loc[index,'Hotel Name']='name not found'
    try:
        address=info['features'][0]['properties']['address_line2']
        print(address)
        hotels_df.loc[index,'Hotel Address']=address

    except:
        print('hotel address not found, skipping')
        hotels_df.loc[index,'Hotel Address']='address not found'
    

hotel name not found, skipping
hotel address not found, skipping
For Blackmans Bay, the nearest hotel is Villa Howden
Howden Road, Hobart TAS 7055, Australia
hotel name not found, skipping
hotel address not found, skipping
For Mount Gambier, the nearest hotel is The Old Mount Gambier Gaol
Langlois Place, Mount Gambier SA 5290, Australia
hotel name not found, skipping
hotel address not found, skipping
hotel name not found, skipping
hotel address not found, skipping
For Lincoln City, the nearest hotel is Inn at Lincoln City
1091 Southeast 1st Street, Lincoln City, OR 97367, United States of America
For Waitangi, the nearest hotel is Hotel Chathams
1 Waitangi Wharf Owenga Road, Waitangi 8942, New Zealand
hotel name not found, skipping
hotel address not found, skipping
For Marsá Maţrūḩ, the nearest hotel is صقور
Alexandria Street, Madara, Marsa Matruh, 51511, Egypt
hotel name not found, skipping
hotel address not found, skipping
For Port Lincoln, the nearest hotel is Boston Hotel
19-21 Kin

For Horta, the nearest hotel is Azoris Faial Garden - Resort Hotel
Rua Cônsul Dabney, 9901-856 Horta, Portugal
For Finisterra, the nearest hotel is Hotel Temático do Banco Azul
Rúa Pescadores, 15155 Fisterra, Spain
For Leiston, the nearest hotel is White Horse Hotel
Station Road, East Suffolk, IP16 4HD, United Kingdom
For Saint-Pierre, the nearest hotel is Tropic Hotel
Rue Auguste Babet, 97410 Saint-Pierre, France
For Salinas, the nearest hotel is The Traveler's Hotel
16, 18 East Gabilan Street, Salinas, CA 93901, United States of America
For Zaghouan, the nearest hotel is Les Hirondelles
شارع الأرض, 1121 Zaghouan Governorate, Tunisia
hotel name not found, skipping
hotel address not found, skipping
hotel name not found, skipping
hotel address not found, skipping
For Mazara del Vallo, the nearest hotel is Greta Hotel
Via Finocchiaro Aprile, 107, 91026 Mazara del Vallo TP, Italy
hotel name not found, skipping
hotel address not found, skipping
For Saint-Jean-de-Monts, the nearest hotel is

For Fort Bragg, the nearest hotel is Airborne Inn Lodging
Bastogne Drive, Hammond Hills, NC 28310, United States of America
hotel name not found, skipping
Djibo, Soum, Sahel, Burkina Faso
For Gwadar, the nearest hotel is Sadaf Resorts
بلوار غربی, Gwadar, Pakistan
For Pampas, the nearest hotel is Hotel Plaza
Jirón Grau, Pampas 09000, Peru


In [83]:
hotels_df.sample(5)

,City Name,Country,City Lat,City Lon,Humidity,Hotel Name,Hotel Address
50,Saurimo,AO,-9.6608,20.3916,99,Complexo Hoteleiro de Saurimo,"Rua da Liberdade, Saurimo, Angola"
41,Biltine,TD,14.5333,20.9167,23,فندق ثلاث نجوم بلتن,"Abeche - Biltine National Road, Biltine, Chad"
107,Timbuktu,ML,16.7735,-3.0074,11,Hotel La Maison,"Rue Gorbeygna, Timbuktu, Mali"
24,Ramnagar,IN,29.4000,79.1167,63,Hotel Corbett Kingdom,"NH309, Rāmnagar - 244715, Uttarakhand, India"
137,Rome,US,43.2128,-75.4557,96,Wingate by Wyndham Rome,"90 Dart Circle, City of Rome, NY 13441, United..."


In [84]:
hotels_df.to_csv('output/nearby_hotels_jupyter_version.csv', index=False)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [2]:
# import saved data in csv if notebook kernel fails
hotels_df=pd.read_csv('output/nearby_hotels_jupyter_version.csv', index_col=None)
hotels_df.sample(5)

,City Name,Country,City Lat,City Lon,Humidity,Hotel Name,Hotel Address
130,Vredendal,ZA,-31.6683,18.5012,66,name not found,address not found
93,Saint-Pierre,RE,-21.3393,55.4781,46,Tropic Hotel,"Rue Auguste Babet, 97410 Saint-Pierre, France"
128,Duki,PK,30.1541,68.5728,31,name not found,address not found
121,Kabare,CD,-2.4683,28.8242,95,Panorama Hotel,"Boulevard Reine Elisabeth, Bukavu, Democratic ..."
176,Morehead City,US,34.7229,-76.7260,86,"Bask Hotel Big Rock Landing, Trademark Collect...","814 Shepard Street, Morehead City, NC 28557, U..."


In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotels_df.hvplot.points(
    "City Lon", 
    "City Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City Name",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)



hvplot.save(map_plot, '../images/hotel_weather_hvplot.html')
hvplot.save(map_plot, '../images/hotel_weather_hvplot.widgets')


In [4]:
# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [City Lon,City Lat]   (City Name,Humidity,Hotel Name,Country)